In [11]:

# debugging platform
import pydra
from pydra.tasks import fsl
from pathlib import Path
from ipyniivue import Niivue
import shutil
import pydra

import nest_asyncio
import shutil
nest_asyncio.apply()


In [12]:
sub_1070230 = Path.cwd() / 'sub-1070230' / 'ses-M000' / 'anat' / 'sub-1070230_ses-M000_T1w.nii.gz' 
sub_seg_107230 = Path.cwd() / 'derivatives'/'manual_segm'/'sub-1070230'/'ses-M000'/'anat'/'sub-1070230_ses-M000_dseg.nii.gz'

sub_1070336 = Path.cwd() / 'sub-1070568/ses-M000/anat/sub-1070568_ses-M000_T1w.nii.gz'
seg_1070336 = Path.cwd() / 'derivatives/manual_segm/sub-1070568/ses-M000/anat/sub-1070568_ses-M000_dseg.nii.gz'



In [13]:
def deface_and_resample_to_template(**kwargs) -> pydra.Workflow:
    from pydra.tasks import fsl
    from pydra.tasks.fsl import fslmaths

    workflow = pydra.Workflow(
        input_spec=[
            "input_image",
            "template_image",
            "template_mask",
            "output_mask",
        ],
        **kwargs,
    )

    workflow.add(
        fsl.FSLReorient2Std(
            name="fslreorient2std",
            input_image=workflow.lzin.input_image,
        )
    )

    reference_image = (
        workflow.lzin.template_image
    )

    workflow.add(
        fsl.FLIRT(
            name="flirt",
            input_image=workflow.fslreorient2std.lzout.output_image,
            reference_image=workflow.lzin.template_image,
            cost_function="mutualinfo",
        )
    )

    workflow.add(
        fslmaths.Mul(
            name="apply_mask",
            input_image=workflow.flirt.lzout.output_image,
            other_image=workflow.lzin.template_mask,
        )
    )

    workflow.set_output({
        "output_image": workflow.apply_mask.lzout.output_image,
        "output_mask": workflow.apply_mask.lzout.output_image,
    })

    return workflow


In [16]:
def deface(**kwargs) -> pydra.Workflow:
    from pydra.tasks import fsl
    from pydra.tasks.fsl import fslmaths

    workflow = pydra.Workflow(
        input_spec=[
            "input_image",
            "template_image",
            "template_mask",
            "output_mask",
        ],
        **kwargs,
    )

    workflow.add(
        fsl.FSLReorient2Std(
            name="fslreorient2std",
            input_image=workflow.lzin.input_image,
        )
    )


    reference_image = (
        workflow.fslreorient2std.lzout.output_image
    )
    
    
    workflow.add(
        fsl.FLIRT(
            name="flirt",
            input_image=workflow.lzin.template_image,
            reference_image=reference_image,
            cost_function="mutualinfo",
        )
    )

    workflow.add(
        fsl.ApplyXFM(
            name="apply_xfm",
            input_image=workflow.lzin.template_mask,
            reference_image=reference_image,
            input_matrix=workflow.flirt.lzout.output_matrix,
        )
    )

    workflow.add(
        fslmaths.Mul(
            name="apply_mask",
            input_image=reference_image,
            other_image=workflow.apply_xfm.lzout.output_image,
        )
    )

    workflow.set_output({
        "output_image": workflow.apply_mask.lzout.output_image,
        "output_mask": workflow.apply_xfm.lzout.output_image,
    })

    return workflow


def deface_and_resample_to_template(**kwargs) -> pydra.Workflow:
    from pydra.tasks import fsl
    from pydra.tasks.fsl import fslmaths

    workflow = pydra.Workflow(
        input_spec=[
            "input_image",
            "template_image",
            "template_mask",
            "output_mask",
            "segm_image",
        ],
        **kwargs,
    )

    workflow.add(
        fsl.FSLReorient2Std(
            name="fslreorient2std",
            input_image=workflow.lzin.input_image,
        )
    )

    reference_image = (
        workflow.lzin.template_image
    )

    workflow.add(
        fsl.FLIRT(
            name="flirt",
            input_image=workflow.fslreorient2std.lzout.output_image,
            reference_image=workflow.lzin.template_image,
            cost_function="mutualinfo",
        )
    )

    workflow.add(
        fslmaths.Mul(
            name="apply_mask",
            input_image=workflow.flirt.lzout.output_image,
            other_image=workflow.lzin.template_mask,
        )
    )
    
    workflow.add(
    fsl.ApplyXFM(
        name="apply_xfm",
        input_image=workflow.lzin.segm_image,
        reference_image=reference_image,
        input_matrix=workflow.flirt.lzout.output_matrix,
    )
)

    workflow.set_output({
        "output_image": workflow.apply_mask.lzout.output_image,
        "output_mask": workflow.apply_mask.lzout.output_image,
        "output_segm": workflow.apply_xfm.lzout.output_image,
    })

    return workflow

In [17]:
Task = deface_and_resample_to_template(input_image=sub_1070230, segm_image=sub_seg_107230, name='deface', 
              template_image='/Users/rosana.eljurdi/Datasets/Lymphoma/defacing-templates/tpl-MNI152Lin_res-01_T1w.nii.gz', 
              template_mask = '/Users/rosana.eljurdi/Datasets/Lymphoma/defacing-templates/tpl-MNI152Lin_res-01_desc-brain_mask.nii.gz',
             )
                
result = Task()

result

Result(output=Output(output_image=PosixPath('/private/var/folders/vl/8gp_6lm16ksg3bnh7__sjzwc00102x/T/tmpq7txyfco/Workflow_0aefdbfa40ca24b8af21aa69a59f4315875370f335d65d9e21485f0b6ba51603/sub-1070230_ses-M000_T1w_r2std_flirt_fslmaths.nii.gz'), output_mask=PosixPath('/private/var/folders/vl/8gp_6lm16ksg3bnh7__sjzwc00102x/T/tmpq7txyfco/Workflow_0aefdbfa40ca24b8af21aa69a59f4315875370f335d65d9e21485f0b6ba51603/sub-1070230_ses-M000_T1w_r2std_flirt_fslmaths.nii.gz'), output_segm=PosixPath('/private/var/folders/vl/8gp_6lm16ksg3bnh7__sjzwc00102x/T/tmpq7txyfco/Workflow_0aefdbfa40ca24b8af21aa69a59f4315875370f335d65d9e21485f0b6ba51603/sub-1070230_ses-M000_dseg_axfm.nii.gz')), runtime=None, errored=False)